In [1]:
from smartpool_config import *

spark = create_spark("smartpool-arquitecture")

print("Spark OK:", spark.version)
print("BASE:", BASE)
print("STATE:", STATE)


:: loading settings :: url = jar:file:/opt/conda/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/jovyan/.ivy2.5.2/cache
The jars for the packages stored in: /home/jovyan/.ivy2.5.2/jars
io.delta#delta-spark_2.13 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a7f00636-8ebc-4527-a777-ad0eee1c571d;1.0
	confs: [default]
	found io.delta#delta-spark_2.13;4.0.0 in central
	found io.delta#delta-storage;4.0.0 in central
	found org.antlr#antlr4-runtime;4.13.1 in central
:: resolution report :: resolve 134ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-spark_2.13;4.0.0 from central in [default]
	io.delta#delta-storage;4.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.13.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf  

Spark OK: 4.0.1
BASE: s3a://spark/medallion
STATE: s3a://spark/medallion/_state


In [2]:
df_pools = (
    spark.read
    .format("jdbc")
    .option("url", JDBC_URL)
    .option("dbtable", "dbo.pools_dim")
    .option("user", JDBC_USER)
    .option("password", JDBC_PASS)
    .option("driver", JDBC_DRIVER)
    .load()
)

print("Pools Dim en SQL Server:")
df_pools.printSchema()
df_pools.show()


Pools Dim en SQL Server:
root
 |-- pool_id: integer (nullable = true)
 |-- pool_name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- volume_liters: integer (nullable = true)
 |-- is_heated: boolean (nullable = true)
 |-- owner_type: string (nullable = true)
 |-- updated_at: timestamp (nullable = true)



+-------+--------------------+--------------------+-------------+---------+-------------+--------------------+
|pool_id|           pool_name|            location|volume_liters|is_heated|   owner_type|          updated_at|
+-------+--------------------+--------------------+-------------+---------+-------------+--------------------+
|      1| Piscina Casa Pueblo|Valdeganga (Albac...|        40000|    false|      private|2026-01-25 18:37:...|
|      2|  Piscina Villa Mila|Valdeganga (Albac...|        70000|    false|      private|2026-01-25 18:37:...|
|      3|Piscina Airbnb Rural|              Cuenca|        35000|    false|       airbnb|2026-01-25 18:37:...|
|      4|Piscina Hotel Centro|              Madrid|        60000|     true|        hotel|2026-01-25 18:37:...|
|      5|Piscina Polidepor...|         Ciudad Real|        80000|     true|sports_center|2026-01-25 18:37:...|
+-------+--------------------+--------------------+-------------+---------+-------------+--------------------+



In [3]:
bronze_pools_path = f"{BRONZE}/pools_dim"

In [4]:
# (
#     df_pools
#     .write
#     .format("delta")
#     .mode("overwrite")
#     .option("overwriteSchema", "true")
#     .save(bronze_pools_path)
# )

# print("Escrito en:", bronze_pools_path)

In [5]:
try:
    df_pools_bronze = (
        spark.read
        .format("delta")
        .load(bronze_pools_path)
    )
    print("Pools Dim en Bronze:")
    df_pools_bronze.show(5, truncate=False)
    df_heated = df_pools_bronze.filter("is_heated = true")
    df_heated.show()
except Exception as e:
    # print(f"Bronze pools_dim aún no existe en {bronze_pools_path}: {e}")
    print(f"Bronze pools_dim aún no existe en {bronze_pools_path}")
    

26/01/25 19:02:45 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


Bronze pools_dim aún no existe en s3a://spark/medallion/bronze/pools_dim


In [6]:
df_maint = (
    spark.read
    .format("jdbc")
    .option("url", JDBC_URL)
    .option("dbtable", "dbo.maintenance_events")
    .option("user", JDBC_USER)
    .option("password", JDBC_PASS)
    .option("driver", JDBC_DRIVER)
    .load()
)

print("Registros en SQL Server de maintenance_events:", df_maint.count())
df_maint.show()
df_maint.printSchema()

Registros en SQL Server de maintenance_events: 14
+---+-------+--------------------+-----------------+------------+--------------+--------------------+--------------------+
| id|pool_id|          event_time|intervention_type|product_type|product_amount|               notes|          updated_at|
+---+-------+--------------------+-----------------+------------+--------------+--------------------+--------------------+
|  1|      1|2026-01-15 01:10:...|         chlorine|    dichloro|         250.0|Tratamiento de ch...|2026-01-25 01:10:...|
|  2|      1|2026-01-18 01:10:...|    ph_correction|       minus|         150.0|Ajuste de pH tras...|2026-01-25 01:10:...|
|  3|      1|2026-01-22 01:10:...|  filter_backwash|        NULL|          NULL|Lavado de filtro ...|2026-01-25 01:10:...|
|  4|      2|2026-01-17 01:10:...|         chlorine|    tricloro|         200.0|Mantenimiento rut...|2026-01-25 01:10:...|
|  5|      2|2026-01-21 01:10:...|           refill|        NULL|        1500.0|Relleno p

In [7]:
bronze_maint_path = f"{BRONZE}/maintenance_events"

In [8]:

# (
#     df_maint
#     .write
#     .format("delta")
#     .mode("overwrite")  # full load inicial
#     .option("overwriteSchema", "true")
#     .save(bronze_maint_path)
# )

# print("Maintenance events escritos en:", bronze_maint_path)

In [ ]:

# Solo lectura de Bronze
try:
    df_maint_bronze = (
        spark.read
        .format("delta")
        .load(bronze_maint_path)
    )

    print("Maintenance events en Bronze:")
    df_maint_bronze.show()
    df_maint_bronze.printSchema()
    print("Registros en Bronze:", df_maint_bronze.count())

    df_join = (
        df_maint_bronze.alias("m")
        .join(df_pools_bronze.alias("p"), F.col("m.pool_id") == F.col("p.pool_id"), "left")
    )
    
    df_join.select(
        "m.id",
        "m.event_time",
        "m.intervention_type",
        "m.product_type",
        "p.pool_name",
        "p.location",
        "p.is_heated"
    ).show(truncate=False)

except Exception as e:
    # print(f"Bronze maintenance_events aún no existe en {bronze_maint_path}: {e}")
    print(f"Bronze maintenance_events aún no existe en {bronze_maint_path}")
    

In [10]:
spark.stop()